In [9]:
#For what seems like ease of use, I am using Google Colab rather than GitHub CodeSpaces
import torch
import torchvision
from torchvision import datasets, models, transforms
from torch.utils.data import Dataset, DataLoader
from torchvision.datasets import ImageFolder
import torch.nn as nn
import torch.optim as optim
import timm

import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

In [23]:
#Of course not everything can be simple so I am working on uploading a local folder of images to Google Colab
import kagglehub

# Download latest version
path = kagglehub.dataset_download("mdwaquarazam/agricultural-crops-image-classification")

print("Path to dataset files:", path)

Path to dataset files: /root/.cache/kagglehub/datasets/mdwaquarazam/agricultural-crops-image-classification/versions/1


In [24]:
PlantDataset = ImageFolder(path)

In [25]:
class PlantDataset(Dataset):
    def __init__(self, data_dir, transform=None):
        self.data = ImageFolder(data_dir, transform=transform)

    def __len__(self):
        return len(self.data)

    def __getitem__(self,idx):
        return self.data[idx]

    @property
    def classes(self):
        return self.data.classes

FileNotFoundError: [Errno 2] No such file or directory: 'PlantDataset'

In [26]:
dataset = PlantDataset(data_dir = '/root/.cache/kagglehub/datasets/mdwaquarazam/agricultural-crops-image-classification/versions/1')

TypeError: 'ImageFolder' object is not callable

In [27]:
len(dataset)

NameError: name 'dataset' is not defined

In [28]:
dataset[9]

NameError: name 'dataset' is not defined

In [29]:
image, label = dataset[9]
image
print(label)

NameError: name 'dataset' is not defined

In [30]:
data_dir = '/root/.cache/kagglehub/datasets/mdwaquarazam/agricultural-crops-image-classification/versions/1'
target_to_class = {v: k for k, v in ImageFolder(data_dir).class_to_idx.items()}
print(target_to_class)

{0: 'Agricultural-crops'}
